### Background

**Module loaders** are the cleanest way of injecting data packages as importable things into Python. We can append to the `sys.meta_path` on importing `t4` (several other packages do this already; `six` does this for example), and write a loader which works with our data packages.

However they are very complicated to understand. To try and help work my way through them I'm preparing this document.


### Organization
To add a module loader to the list, just append it to `sys.meta_path`. Easy.

Module loaders themselves consist of two pieces of class machinery: a `<>Finder` and a `<>Loader`.

The finder is what sits on `sys.meta_path` and it is responsible for finding the code corresponding with a given package. Its one required method is `find_spec`, which takes as input a `fullname` plus some other stuff. It is supposed to return a `ModuleSpec` class. The `ModuleSpec` class is via `importlib.machinery` and must be parameterized with two values: a `name` (a string for the fully-qualified name of the module) and a loader.

That `ModuleSpec` class then executes the `create_module` method on the loader, presenting itself as the sole argument to `create_module` (the `spec` argument). As far as I can tell it does not do any work in between its own class instatiation and executing the loader code.

The `create_module` routine is expected to create a module that is executable. A module is regular class with vanilla parameters which more or less preserves all of the things passed down the inheritance hierarchy so far:

    {'__name__': 't4.data', '__doc__': None, '__package__': 't4', '__loader__': <loader.DataPackageLoader object at 0x104100588>, '__spec__': ModuleSpec(name='t4.data', loader=<loader.DataPackageLoader object at 0x104100588>)}

If this method returns `None`, the default module creation semantics will happen. Overwriting this method just means implementing the module construct yourself.

Finally, `exec_module` is expected to execute the module. This means running e.g. `exec(code, module.__dict__`), where `code` is some code object or valid Python string, and then returning the module. All arguments to `exec` besides the first act as a global environment for the code execution, hence the `__dict__` here. How you generate that code object is up to you; the `module` object is just providing you the "right" built-ins for Python to know what to do with your module.


### Submodules
All of the above is sufficient for implementing a top-level module. However, to implement submodules on that module there is one additional thing you must do: provide a `__path__` value in the module.

The `__path__` value on a module replaces the `sys.path` variables when searching through a module tree. In the case of code modules, this makes sense: we are no longer interested in _any_ file whose names matches module signature; we now want _specifically_ files whose names match the next string segment which are located underneat the previous module.

Since we can define our module object with arbitrary code, we can have "virtual" modules that don't correspond to any file on disc. In such cases we cannot meaningfully populate `__path__`.

However, _failing_ to populate `__path__` causes the import to fail, presumably because the loaders that rely on it don't gracefully handle its absence.

We can specify a fake `__path__`. That `__path__` *will* be scanned by the loaders that preface our loader in the loader order, however, so it better not point to anything that will accidentally get loaded by another loader.